<a href="https://colab.research.google.com/github/Aigulchiks/My_projects_Rus/blob/main/web_app_in_panell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Загрузка библиотек и данных:

In [45]:
!pip install -q hvplot
!pip install pyngrok --quiet
!pip install panel==0.12.6 -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyngrok import ngrok
import holoviews as hv
import requests
import json
import pandas as pd
import numpy as np
import panel as pn
pn.extension()
from panel.interact import interact
hv.extension('bokeh')
import holoviews.plotting.bokeh
import hvplot.pandas
pn.extension(sizing_mode = 'stretch_width')

Данные биткоина: 

In [46]:
url = "http://api.coincap.io/v2/assets/bitcoin/history?interval=d1"
payload = {}
headers= {}

response = requests.request("GET", url, headers=headers, data = payload)

json_data = json.loads(response.text.encode('utf8'))

bitcoin_data = json_data["data"]
bitcoin_data = pd.DataFrame(bitcoin_data)
bitcoin_data['time'] = pd.to_datetime(bitcoin_data['time'], unit='ms')
bitcoin_data['priceUsd'] = pd.to_numeric(bitcoin_data['priceUsd'], errors='coerce').fillna(0, downcast='infer')
bitcoin_data = pd.DataFrame(bitcoin_data, columns=['time', 'priceUsd'])
bitcoin_data ['asset'] = 'bitcoin'

Данные эфира: 

In [47]:
url = "http://api.coincap.io/v2/assets/ethereum/history?interval=d1"
payload = {}
headers= {}

response = requests.request("GET", url, headers=headers, data = payload)

json_data = json.loads(response.text.encode('utf8'))

ethereum_data = json_data["data"]
ethereum_data = pd.DataFrame(ethereum_data)
ethereum_data ['time'] = pd.to_datetime(ethereum_data ['time'], unit='ms')
ethereum_data ['priceUsd'] = pd.to_numeric(ethereum_data ['priceUsd'], errors='coerce').fillna(0, downcast='infer')
ethereum_data  = pd.DataFrame(ethereum_data , columns=['time', 'priceUsd'])
ethereum_data ['asset'] = 'ethereum'

Единый дата-сет: 

In [48]:
idf = pd.concat([bitcoin_data, ethereum_data], axis=0)

In [49]:
asset = ['bitcoin', 'ethereum']
select = pn.widgets.Select(name='Select', options=asset)

In [ ]:
#Для запуска в первый раз нужно выбрать дату в полноэкранном режиме ячейки (можно открыть правой кнопкой мыши).
start = pn.widgets.DatePicker(name='From')
start

DatePicker(name='From', sizing_mode='stretch_width')

In [ ]:
#Для запуска в первый раз нужно выбрать дату в полноэкранном режиме ячейки (можно открыть правой кнопкой мыши).
end = pn.widgets.DatePicker(name='to')
end

DatePicker(name='to', sizing_mode='stretch_width')

Интерактивный график 

In [50]:
def create_plot(symbol, start, end):
    pipeline = (
    idf[
        (idf['asset'] == select.value) &
        (idf['time'] >= pd.to_datetime(start)) & (idf['time'] <= pd.to_datetime(end)) 
        
    ]
    [['time','priceUsd']]
    .reset_index()
    .reset_index(drop=True)
)
    pipeline ['time'] = pd.to_datetime(pipeline ['time']).dt.date
    plot = pipeline.hvplot.bar(x = 'time', y= 'priceUsd', line_width=2, title="Price Dynamics", rot=60)
    return  plot


plot = pn.interact(create_plot, symbol=select, start = start, end = end)
plot

Column(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Select(options=['bitcoin', 'ethereum'], sizing_mode='stretch_width', value='bitcoin')
        [1] DatePicker(name='From', sizing_mode='stretch_width', value=datetime.date(2023, 1, 1))
        [2] DatePicker(name='to', sizing_mode='stretch_width', value=datetime.date(2023, 1, 9))
    [1] Row(sizing_mode='stretch_width')
        [0] HoloViews(Bars, name='interactive15971', sizing_mode='stretch_width')

Создаём лэйаут и делимся приложением:

In [57]:
template = pn.template.MaterialTemplate(
        title='Assets',
        main=pn.Row(pn.Column(plot)))

In [58]:
server=pn.serve({"": template}, port=80, address="0.0.0.0", threaded=True, websocket_origin="*")

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.8/dist-packages/panel/io/server.py", line 771, in run
    bokeh_server = target(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/panel/io/server.py", line 722, in get_server
    server = Server(apps, port=port, **opts)
  File "/usr/local/lib/python3.8/dist-packages/bokeh/server/server.py", line 426, in __init__
    sockets, self._port = bind_sockets(opts.address, opts.port)
  File "/usr/local/lib/python3.8/dist-packages/bokeh/server/util.py", line 73, in bind_sockets
    ss = netutil.bind_sockets(port=port or 0, address=address)
  File "/usr/local/lib/python3.8/dist-packages/tornado/netutil.py", line 174, in bind_sockets
    sock.bind(sockaddr)
OSError: [Errno 98] Address already in use


In [59]:
!curl http://localhost:80

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Assets</title>

<link rel="icon" href="/static/extensions/panel/icons/favicon.ico" type="">
<meta name="name" content="Assets">






    <link rel="stylesheet" href="static/extensions/panel/css/json.css" type="text/css" />
    <link rel="stylesheet" href="static/extensions/panel/css/dataframe.css" type="text/css" />
    <link rel="stylesheet" href="static/extensions/panel/css/card.css" type="text/css" />
    <link rel="stylesheet" href="static/extensions/panel/css/widgets.css" type="text/css" />
    <link rel="stylesheet" href="static/extensions/panel/css/alerts.css" type="text/css" />
    <link rel="stylesheet" href="static/extensions/panel/css/markdown.css" type="text/css" />
    <link rel="stylesheet" href="static/extensions/panel/css/loading.css" type="text/css" />
    <style>
        
    .bk.pn-loading.arcs:before {
      background-image: url("");
      max-height: 400px;
    }
    
    </style>

<s

In [60]:
#@title токен
NGROK_AUTH_TOKEN = "2L0p6HxxSUi1QJWhQehgHhh6MYO_2D8MnHHvqaonqUSoQgkRt"

Финальная ссылка после NgrokTunnel:

In [61]:
ngrok.kill()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(addr="80")
print("Tracking URL:", public_url)

Tracking URL: NgrokTunnel: "http://907d-34-69-179-200.ngrok.io" -> "http://localhost:80"


In [62]:
#!curl http://907d-34-69-179-200.ngrok.io 
#стоп сервер 